In [ ]:
using MXNet
using ProgressMeter
using Plotly
using JLD2

In [ ]:
mlp = @mx.chain mx.Variable(:data)             =>
mx.FullyConnected(name=:fc1, num_hidden=128) =>
mx.Activation(name=:relu1, act_type=:relu)   =>
mx.FullyConnected(name=:fc2, num_hidden=64)  =>
mx.Activation(name=:relu2, act_type=:relu)   =>
mx.FullyConnected(name=:fc3, num_hidden=10)  =>
mx.SoftmaxOutput(name=:softmax)

# data provider
batch_size = 100
train_provider, eval_provider = get_mnist_providers(batch_size)

# setup model
model = mx.FeedForward(mlp, context=mx.gpu(0))

# optimizer
optimizer = mx.SGD(lr=0.01, momentum=0.9)

# fit parameters
mx.fit(model, optimizer, train_provider, eval_data=eval_provider, n_epoch=10)

In [ ]:
function perturbweights(weightDict::Dict, perturb::Dict,t::Array{Float32,1})
    tl = length(t)
    perturbed = [Dict{Symbol,MXNet.mx.NDArray}() for i in 1:tl]
    for (arg, array) in weightDict
        for i in 1:tl
            push!(perturbed[i], arg => (1-t[i]) .* weightDict[arg] .+ t[i]*copy(norm(weightDict[arg]))[1]/copy(norm(perturb[arg]))[1] .* perturb[arg])
        end
    end
    return perturbed::Array{Dict{Symbol,MXNet.mx.NDArray},1}
end

function perturb(weightDict::Dict)
    perturb = Dict()
    for (arg, array) in weightDict
        push!(perturb,arg => mx.NDArray(rand(Float32,size(weightDict[arg]))))
    end
    return perturb::Dict
end

function perturbweights2dbilinear2(θ0::Dict,θ1::Dict,θ2::Dict,θ3::Dict,t::Array{Float32,1}) # θ3 is the current minimal model is at
    tl = length(t)
    α,β = t,t
    perturbed = [[Dict{Symbol,MXNet.mx.NDArray}() for i in 1:tl] for i in 1:tl]
    @showprogress for (arg, array) in θ3
        for i in 1:tl
            for j in 1:tl
                #ξ = α[i] .* θ0[arg] .+ (1-α[i]) .* θ1[arg]
                ϕ = α[i]*copy(norm(θ3[arg]))[1]/copy(norm(θ2[arg]))[1] .* θ2[arg] .+ (1-α[i]) .* θ3[arg]
                #θ = β[j] .* ξ .+ (1-β[j]) .* ϕ
                θ = β[j]*copy(norm(θ3[arg]))[1]/copy(norm(θ0[arg]))[1] .* θ0[arg] .+
                            (1-β[j])*copy(norm(θ3[arg]))[1]/copy(norm(ϕ))[1] .* ϕ
                push!(perturbed[i][j], arg => θ )
            end
        end
    end
    return perturbed::Array{Array{Dict{Symbol,MXNet.mx.NDArray},1},1}
end

In [ ]:
minimal = model.arg_params

In [ ]:
# 1d perturb
perturbvector = perturb(minimal)
t = [Float32(t) for t in -2:0.1:2]
perturbed = perturbweights(minimal,perturbvector,t);

#get loss
acearray = []
acuarray = []
@showprogress for m in perturbed
    model.arg_params = m
    acu,ace = averagedcrossentropy(model, eval_provider);
    push!(acearray,ace)
    push!(acuarray,acu)
end

In [ ]:
Plotly.plot(t,acearray)

In [ ]:
θ0,θ1,θ2 = perturb(minimal),perturb(minimal),perturb(minimal)
θ3 = minimal
t = [Float32(t) for t in -2:0.1:2]
perturbed = perturbweights2dbilinear2(θ0,θ1,θ2,θ3,t);

# 2d perturb
tl = length(t)
acearray2d = Array{Float32,2}(tl,tl)
acuarray2d = Array{Float32,2}(tl,tl)
@showprogress for i in 1:tl
    for j in 1:tl
        model.arg_params = perturbed[i][j]
        acu,ace = averagedcrossentropy(model, eval_provider);
        acearray2d[i,j] = ace
        acuarray2d[i,j] = acu
    end
end

In [ ]:
#@save "acearray2d-mlp.jld2" acearray2d
@load "acearray2d-mlp.jld2" acearray2d

In [ ]:
trace = Plotly.surface(z=log10.(acearray2d))
Plotly.plot(trace,Layout(autosize=false))

In [ ]:
# input
data = mx.Variable(:data)

# first conv
conv1 = @mx.chain mx.Convolution(data, kernel=(5,5), num_filter=20)  =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# second conv
conv2 = @mx.chain mx.Convolution(conv1, kernel=(5,5), num_filter=50) =>
                  mx.Activation(act_type=:tanh) =>
                  mx.Pooling(pool_type=:max, kernel=(2,2), stride=(2,2))

# first fully-connected
fc1   = @mx.chain mx.Flatten(conv2) =>
                  mx.FullyConnected(num_hidden=500) =>
                  mx.Activation(act_type=:tanh)

# second fully-connected
fc2   = mx.FullyConnected(fc1, num_hidden=10)

# softmax loss
lenet = mx.SoftmaxOutput(fc2, name=:softmax)


#--------------------------------------------------------------------------------
# load data
batch_size = 100
train_provider, eval_provider = get_mnist_providers(batch_size; flat=false)

#--------------------------------------------------------------------------------
# fit model
model = mx.FeedForward(lenet, context=mx.gpu(0))

# optimizer
optimizer = mx.SGD(lr=0.01,momentum=0.9)

# fit parameters
mx.fit(model, optimizer, train_provider, n_epoch=10, eval_data=eval_provider)

In [ ]:
minimal = model.arg_params

In [ ]:
# 1d perturb
perturbvector = perturb(minimal)
t = [Float32(t) for t in -2:0.1:2]
perturbed = perturbweights(minimal,perturbvector,t);

#get loss
acearray = []
acuarray = []
@showprogress for m in perturbed
    model.arg_params = m
    acu,ace = averagedcrossentropy(model, eval_provider);
    push!(acearray,ace)
    push!(acuarray,acu)
end

In [ ]:
Plotly.plot(t,acearray)

In [ ]:
θ0,θ1,θ2 = perturb(minimal),perturb(minimal),perturb(minimal)
θ3 = minimal
t = [Float32(t) for t in -2:0.1:2]
perturbed = perturbweights2dbilinear2(θ0,θ1,θ2,θ3,t);

# 2d perturb
tl = length(t)
acearray2d = Array{Float32,2}(tl,tl)
acuarray2d = Array{Float32,2}(tl,tl)
@showprogress for i in 1:tl
    for j in 1:tl
        model.arg_params = perturbed[i][j]
        acu,ace = averagedcrossentropy(model, eval_provider);
        acearray2d[i,j] = ace
        acuarray2d[i,j] = acu
    end
end

In [ ]:
#@save "acearray2d-conv.jld2" acearray2d
@load "acearray2d-conv.jld2" acearray2d

In [ ]:
trace = Plotly.surface(z=log10.(acearray2d))
Plotly.plot(trace,Layout(autosize=false))

In [ ]:
function get_mnist_providers(batch_size::Int; data_name=:data, label_name=:softmax_label, flat=true)
  filenames = mx.get_mnist_ubyte()
  train_provider = mx.MNISTProvider(image=filenames[:train_data],
                                    label=filenames[:train_label],
                                    data_name=data_name, label_name=label_name,
                                    batch_size=batch_size, shuffle=true, flat=flat, silent=true)
  eval_provider = mx.MNISTProvider(image=filenames[:test_data],
                                   label=filenames[:test_label],
                                   data_name=data_name, label_name=label_name,
                                   batch_size=batch_size, shuffle=false, flat=flat, silent=true)

  return (train_provider, eval_provider)
end
function averagedcrossentropy(model, eval_provider)
    probs = mx.predict(model, eval_provider)
    labels = Array[]
    for batch in eval_provider
    push!(labels, copy(mx.get(eval_provider, batch, :softmax_label)))
    end
    labels = cat(1, labels...)

    # Now we use compute the accuracy
    correct = 0
    # ace
    acetotal = 0
    for i = 1:length(labels)
        # labels are 0...9
        if indmax(probs[:,i]) == labels[i]+1
          correct += 1
        end
        acetotal += log2(probs[Int(labels[i]+1),i])
    end
    ace = -acetotal/length(labels)
    accuracy = 100correct/length(labels)
    #println(mx.format("Accuracy on eval set: {1:.2f}%", accuracy))
    return accuracy,ace
end


function save_checkpoint(sym :: mx.SymbolicNode, arg_params :: Dict{Base.Symbol, mx.NDArray},
                         aux_params :: Dict{Base.Symbol, mx.NDArray}, filename :: AbstractString,epochs::String)
  mx.save("$filename-symbol.json", sym)
  save_dict = merge(Dict{Base.Symbol, mx.NDArray}(map((x) -> Symbol("arg:$(x[1])") => x[2], arg_params)),
                    Dict{Base.Symbol, mx.NDArray}(map((x) -> Symbol("aux:$(x[1])") => x[2], aux_params)))
  mx.save("$filename-$epochs\.params", save_dict)
  info("Saved checkpoint to '$filename'")
end